# Explicacion del codigo, del primero avance

## Descripcion

En este apartado se colocara una revision del codigo, para verificar que se entiende, que aspectos se deben estudiar y como podria hacerlo, empezaremos por secciones

### Primera PARTE, Importacion de Librerias

In [ ]:
import os # la herramienta de matematica con muchas funciones de esta indole
import math #Importa la libreria de gestion de archivos "os" 
import numpy as np # Libreria para calculos en dataframes
import tensorflow as tf # importe de libreria tensorflow de deep learning
import matplotlib.pyplot as plt # Libreria que permite hacer graficos mas adecuados
from mlutils import setup_gpu
setup_gpu()

tf.keras.utils.set_random_seed(0) # en esta parte se define una semilla, esta permite conservar los mismos valores siempre que se haga la ejecucion

2025-12-27 11:38:15.693681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-27 11:38:16.072810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-27 11:38:16.893193: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/mech/tf_gpu/venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: Futu

ModuleNotFoundError: No module named 'gpu_setup'

### Segunda PARTE, Definicion de TRIAL con estructura fisica, Planteamiento Fisico

En esta parte el interes es crear una clase que permita a la red usar una funcion a diferenciar mas acorde con el problema fisico de pozo de potencial, es decir que la ayude con la estructura de la prediccion.

In [ ]:
def trig_nodal_factor(x,n): ## Esta funcion debe recibir los puntos espaciales y los modos 

    n_tf = tf.cast(n , x.dtype) ## aca usamos un casting para que la version de tensorflow no tenga problemas, convertimos el tipo de dato de n, al tipo de dato que tenga x, tambien  cabe aclarar que el casting de tensorflow, recibe dos cosas, la variable y el tipo de dato
    pi_tf = tf.constant (math.pi, dtype = x.dytpe) # garantizamos que esta constante sea Tensorflow

    # Ahora procederemos a determinar el factor trigonometrico 

    sin_fijo = tf.sin(pi_tf * x)
    sin_n =  tf.sin(n_tf*pi_tf*x)

    # Para evitar divisiones por cero, se hace un ajuste, en donde garanticemos que esto nunca pase

    ratio = sin_n / (sin_fijo + 1e-12) # Ese parche garantiza que nunca se divida por cero

    # tf.where, funciona de la siguiente forma tf.where(condición, valor_si_verdadero, valor_si_falso)



    return tf.where(tf.abs(sin_fijo) < 1e-6, n_tf, ratio) # Esto genera una condicion, si el valor del seno del denominador es cercano a cero, use el valor de n,sino use el general, por el limite de x cuando tiende a cero


# =================================== Construccion de la Red ==============================#
# Algo importante , es que aqui no se define el entrenamiento simplemente se construye el modelo

def make_net(n = 1, hidden = 64, use_sine = True): #la funcion de construccion de la red, recibe 3 parametros, el modo, la cantidad de neuronas y finalmente un booleano de si debe usar sin o no

    x_in = tf.keras.Input(shape = (1,), dtype = tf.float32) # en esta linea se determina los parametros de entrada, en este caso la entrada de x, el comando de input, requiere la forma "shape" y el tipo de dato

## Ahora vienen las condiciones ## 

## ===== Caso 1 , activacion del seno ===== ##

    if use_sine:
    # Esta es la primera capa oculta de la red, va a recibir neuronas, funcion de activacion que se aplicara todas las neuronas, como se inicializa los pesos, se hace a traves del kernel_inizialiter
        z = tf.keras.layers.Dense(
            hidden,
            activation = tf.math.sin,
            kernel_initializer = "glorot_uniform", # aqui se inicializan los pesos, en paticular glorot_uniform usa pesos de una distribucion uniforme
            bias_initializer = "zeros"
        )(x_in)
        # ahora vamos con las segunda capa
        z = tf.keras.layers.Dense(
            hidden,
            activation = tf.math.sin,
            kernel_initializer = "glorot_uniform",
            bias_initializer = "zeros"
        )(z)

    else : 
        z = tf.keras.layers.Dense(
            hidden,
            activation = "tanh",
            kernel_initializer = "glorot_uniform",
            bias_initializer = "zeros"
        )(x_in)

        z = tf.keras.layers.Dense(
            hidden,
            activation = "tanh",
            kernel_initializer = "glorot_uniform",
            bias_initializer = "zeros"

        )(z)

    ## ============ Definicion de la salida =================## esta capa es Nθ​

    out = tf.keras.layers.Dense(
        1, # aca es uno porque es una salida
        activation = None, # en la salida no aplica una funcion de activacion
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros"
    )(z) # recibe a z, para la salida

    #
    F = tf.keras.layers.Lambda(lambda x: trig_nodal_factor(x,n))(x_in) # Lambda es una capa sin pesos que sirve para meter matemáticas hechas a mano dentro del modelo.

    psi = tf.keras.layers.Lambda(
        lambda inputs: inputs[0] * (1-inputs[0]) * inputs[1] * inputs[2]
    ) (x_in, F, out)

    return tf.keras.model(inputs = x_in, outputs = psi)



### Luego de construida la red, se procede a revisar la diferenciacion, es decir la derivada, en particular la segunda derivada

In [ ]:
def second_derivate(model,x):   # la funcion recibe el modelo construido y el valor x a diferenciar con la derivada

    x = tf.convert_to_tensor(x); x = tf.reshape(x, (-1,1))

 # Esta línea cumple dos funciones importantes:
# (1) Convierte lo que entre como x en un tensor de TensorFlow.
# (2) Mediante reshape((-1,1)) garantiza que x tenga la forma
#     (numero_de_muestras, numero_de_variables), que es la estructura
#     esperada por las capas densas de la red.
# El -1 indica que TensorFlow calcula automáticamente el número de muestras,
# mientras que el 1 fija que solo hay una variable independiente (x).

    with tf.GradientTape(persistent= True) as t2 :
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x) # watch(x) le dice a GradientTape que x es la variable independiente respecto a la cual se van a calcular derivadas.
            psi = model(x)
        psi_x = t1.gradient(psi, x)
    psi_xx = t2.gradient(psi_x, x)

    return psi, psi_xx



 # Se abre una cinta de gradientes externa (persistent=True) que permite
# calcular derivadas de orden superior. Esta cinta registrará todas las
# operaciones necesarias para obtener la segunda derivada respecto a x.
           


In [ ]:
def second_derivate(x):

    x = tf.convert_to_tensor(x, dtype=tf.float32)
    x = tf.reshape(x, (-1,1))

    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x)
            psi = 3*x**3
        psi_x = t1.gradient(psi, x)

    psi_xx = t2.gradient(psi_x, x)
    tf.print("psi =", psi)
    tf.print("psi_xx =", psi_xx)

    
    return psi, psi_xx


In [12]:
second_derivate(3)



tf.Tensor([[54.]], shape=(1, 1), dtype=float32)


(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[81.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[54.]], dtype=float32)>)